In [1]:
import numpy as np
import pandas as pd
from SERGIO.sergio import sergio

# Simulate Clean Data _ Steady-State Simulation

In [2]:
sim = sergio(number_genes=5, number_bins = 4, number_sc = 100, noise_params = 1, decays=0.8, sampling_state=3, noise_type='dpd')
sim.build_graph(input_file_taregts ='data_sets/jens_grn/grn.txt', input_file_regs='data_sets/jens_grn/regulators_cid.txt', shared_coop_state=2)
sim.simulate()

expr = sim.getExpressions()
expr_clean = np.concatenate(expr, axis = 1)


Start simulating new level
There are 1 genes to simulate in this layer
Done with current level
Start simulating new level
There are 1 genes to simulate in this layer
Done with current level
Start simulating new level
There are 1 genes to simulate in this layer
Done with current level
Start simulating new level
There are 2 genes to simulate in this layer
Done with current level


# Add Technical Noise _ Steady-State Simulations

In [ ]:
"""
Add outlier genes
"""
expr_O = sim.outlier_effect(expr, outlier_prob = 0.01, mean = 0.8, scale = 1)

"""
Add Library Size Effect
"""
libFactor, expr_O_L = sim.lib_size_effect(expr_O, mean = 4.6, scale = 0.4)

"""
Add Dropouts
"""
binary_ind = sim.dropout_indicator(expr_O_L, shape = 6.5, percentile = 82)
expr_O_L_D = np.multiply(binary_ind, expr_O_L)

"""
Convert to UMI count
"""
count_matrix = sim.convert_to_UMIcounts(expr_O_L_D)

"""
Make a 2d gene expression matrix
"""
count_matrix = np.concatenate(count_matrix, axis = 1)

/oscar/home/jchen397/test_sergio/SERGIO/SERGIO/sergio.py:911: RuntimeWarning: divide by zero encountered in divide
  binFactors = np.true_divide(binFactors, normalizFactors)
/oscar/home/jchen397/test_sergio/SERGIO/SERGIO/sergio.py:915: RuntimeWarning: invalid value encountered in multiply
  ret_data.append(np.multiply(binExprMatrix, binFactors))


prob_ber contains NaNs: True
Min value: nan, Max value: nan


ValueError: p < 0, p > 1 or p contains NaNs

# Simulate Clean Data _ differentiation Simulation

In [2]:
# !head data_sets/jens_grn/differentiation.tab

df = pd.read_csv('data_sets/jens_grn/differentiation.tab', delim_whitespace=True, header=None, index_col=None)
bMat = df.values
print(bMat.shape)
#rows and cols correspond to genes and cells (just verify)
# 100 x (6 x 300 )
#  100 genes, 6 cell types, 6 bins of 300 cells per cell type
# sim = sergio(number_genes=100, number_bins = 6, number_sc = 300, noise_params = 0.2, decays=0.8, sampling_state = 1, noise_params_splice = 0.07, noise_type='dpd', dynamics=True, bifurcation_matrix= bMat)
# sim.build_graph(input_file_taregts ='data_sets/De-noised_100G_6T_300cPerT_dynamics_7_DS6/Interaction_cID_7.txt', input_file_regs='data_sets/De-noised_100G_6T_300cPerT_dynamics_7_DS6/Regs_cID_7.txt', shared_coop_state=2)

sim = sergio(number_genes=5, number_bins = 3, number_sc = 100, noise_params = 1, decays=0.8, sampling_state=3, noise_type='dpd')

sim.build_graph(input_file_taregts ='data_sets/jens_grn/grn.txt', input_file_regs='data_sets/jens_grn/regulators_cid.txt', shared_coop_state=2)

sim.simulate_dynamics()
# final ct matrix is the sum of U and S
#spliced matrix, unspliced, sum of both, pseudotime (probs just a vector bc each element in pseudotime is 1d scalar for 1 cell)
exprU, exprS = sim.getExpressions_dynamics()
exprU_clean = np.concatenate(exprU, axis = 1)
print(exprU_clean)
print(exprU_clean.shape)

exprS_clean = np.concatenate(exprS, axis = 1)
print(f"exprU_clean shape: {exprU_clean.shape}")
print(f"exprS_clean shape: {exprS_clean.shape}")

# Print the number of columns for each row
for i, row in enumerate(exprU_clean):
    print(f"Row {i} of exprU_clean has {len(row)} columns")

for i, row in enumerate(exprS_clean):
    print(f"Row {i} of exprS_clean has {len(row)} columns")

# Optionally save to text files
np.savetxt('exprU_clean.txt', exprU_clean, delimiter=' ', fmt='%f')
np.savetxt('exprS_clean.txt', exprS_clean, delimiter=' ', fmt='%f')

/tmp/ipykernel_2061176/1147402302.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv('data_sets/jens_grn/differentiation.tab', delim_whitespace=True, header=None, index_col=None)


(3, 3)
bindict


KeyError: 0

# Add Technical Noise _ differentiation Simulations

In [ ]:
"""
Add outlier genes
"""
exprU_O, exprS_O = sim.outlier_effect_dynamics(exprU, exprS, outlier_prob = 0.01, mean = 0.8, scale = 1)

"""
Add Library Size Effect
"""
libFactor, exprU_O_L, exprS_O_L = sim.lib_size_effect_dynamics(exprU_O, exprS_O, mean = 4.6, scale = 0.4)

"""
Add Dropouts
"""
binary_indU, binary_indS = sim.dropout_indicator_dynamics(exprU_O_L, exprS_O_L, shape = 6.5, percentile = 82)
exprU_O_L_D = np.multiply(binary_indU, exprU_O_L)
exprS_O_L_D = np.multiply(binary_indS, exprS_O_L)

"""
Convert to UMI count
"""
count_matrix_U, count_matrix_S = sim.convert_to_UMIcounts_dynamics(exprU_O_L_D, exprS_O_L_D)

"""
Make 2d spliced and unspliced expression matrices
"""
count_matrix_U = np.concatenate(count_matrix_U, axis = 1)
count_matrix_S = np.concatenate(count_matrix_S, axis = 1)

In [ ]:
sim.calculate_ssConc_()